In [2]:
import torch
import numpy as np
import pandas as pd
import glob
import os
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision.io import read_image 
from torchvision import datasets, models, transforms
base_dir = '../dataset/MN_015_HH_01/MN_015_HH_01/handheld0/'

In [3]:
import torchvision.models as models
vgg = models.vgg16(pretrained=True)


In [4]:
df = pd.read_csv((base_dir+"cam0/"+"data.csv"))
# all_image_df = pd.DataFrame()
# all_image_df['image'] = df['image']

# all_img_df['key_id'] = all_img_df['path'].map(lambda x: splitext(os.path.basename(x))[0])
# all_img_df['car_id'] = all_img_df['key_id'].map(lambda x: x.split('_')[0])
# all_img_df['mask_path'] = all_img_df['path'].map(lambda x: x.replace('train', 'train_masks').replace('.jpg', '_mask.gif'))
# all_img_df['exists'] = all_img_df['mask_path'].map(os.path.exists)
# print(all_img_df['exists'].value_counts())
# all_img_df.sample(3)


In [5]:
#combine all possible 3 touple, 2 ref 1 query

In [20]:
class OivioDataset(Dataset):
    def __init__(self,annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)['filename']
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
    def __len__(self):
        return len(self.img_labels)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels[idx])
        image = read_image(img_path)
        label = self.img_labels[idx]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

    


In [21]:
dataset = OivioDataset("../dataset/MN_015_HH_01/MN_015_HH_01/handheld0/cam0/data.csv",base_dir+"cam0/data",transform=transforms.Resize([224,224]), target_transform=None)

In [22]:
dataloader = DataLoader(dataset, batch_size=16, shuffle=True, num_workers=4)

In [82]:
# del vgg.classifier
vgg.avgpool = torch.nn.Identity()
vgg.features[30] = torch.nn.Identity()
# print(vgg)

In [24]:
for param in vgg.features:
    param.requires_grad = False

In [83]:
class Upsampling(torch.nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, padding=0):
        super(Upsampling, self).__init__()
        # print("in_channels: ",in_channels)
        # print("out_channels: ",out_channels)
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.upsample = torch.nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
                                     
    def forward(self, x1,x2,final = False):
        x1up = self.upsample(x1)
        # print("x1up?: ",x1up.shape)
        # print("x2.shape: ",x2.shape)
        if x1up.shape != x2.shape:
            x1up = torch.nn.functional.pad(x1up, (0, x2.shape[3] - x1up.shape[3], 0, x2.shape[2] - x1up.shape[2]))

        x = torch.cat([x2, x1up], dim=1)
        if final:
            output= torch.nn.Sequential(torch.nn.Conv2d(64+17, 17, kernel_size= 2, padding= 0,stride=1),
                                     torch.nn.BatchNorm2d(17),
                                     torch.nn.ReLU(True),
                                     
                                     torch.nn.Conv2d(17,17, kernel_size=1, padding= 0,stride=1),
                                     torch.nn.Sigmoid())(x)
            return output
        output = torch.nn.Sequential(torch.nn.Conv2d(self.in_channels, self.out_channels, kernel_size= 2, padding= 0,stride=1),
                                     torch.nn.BatchNorm2d(self.out_channels),
                                     torch.nn.ReLU(True),
                                     
                                     torch.nn.Conv2d(self.out_channels, self.out_channels, kernel_size= 2, padding= 0,stride=1),
                                     torch.nn.BatchNorm2d(self.out_channels),
                                     torch.nn.ReLU(True))(x)
        

        return output

In [84]:
class UNet(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels

        self.vgg = vgg

        self.up_1 = Upsampling(1024, 17)
        self.up_2 = Upsampling(256+17, 17)
        self.up_3 = Upsampling(128+17, 17)
        self.up_4 = Upsampling(64+17, 17)
        # self.up_4 =
        # self.outputs = LastConv(64+17, 17)
    
    def forward(self,x):
        x1 = self.vgg.features[0:4](x)
        x2 = self.vgg.features[4:9](x1)
        x3 = self.vgg.features[9:16](x2)
        x4 = self.vgg.features[16:23](x3)
        x5 = self.vgg.features[23:30](x4)
        # print(x1.shape)
        # print(x2.shape)
        # print(x3.shape)
        # print(x4.shape)
        # print(x5.shape)

        x6 = self.up_1(x5,x4)
        x7 = self.up_2(x6,x3)
        x8 = self.up_3(x7,x2)
        x9 = self.up_4(x8,x1,True)

        # (F3,S3),(F2,S2),(F1,S1),(F0,S0)
        return (x6[:15],x6[15:]),(x7[:15],x7[15:]),(x8[:15],x8[16:]),(x9[:15],x9[16:])
        


In [85]:
net = UNet(1,17)

In [86]:
# for (x,y) in dataloader:
#     print(x.shape)
#     print(y.shape)
# print(dataset[0][0].)
(F3,S3),(F2,S2),(F1,S1),(F0,S0) = net(dataset[0][0].float().repeat(3,1,1).unsqueeze(0))

In [ ]:
# for each 